In [ ]:
import tkinter as tk
from tkinter import font
from PIL import ImageTk, Image
from moviepy.editor import VideoFileClip
import tkinter.messagebox as messagebox
import webbrowser
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    return pose

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

colors = [(245,117,16), (117,245,16), (16,117,245)]
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities
model = keras.models.load_model('action')
sequence = []
sentence = []
predictions = []
threshold = 0.5
actions = np.array(['child pose', 'elbow flexion', 'leg curl'])
fps_start_time = time.time()
fps_frame_count = 0



def begin():
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-100:]

            if len(sequence) == 100:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, actions, image, colors)
            
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
            fps_frame_count += 1
            fps_end_time = time.time()
            fps_time_diff = fps_end_time - fps_start_time
            if fps_time_diff >= 1.0:  # Update FPS value every 1 second
                fps = fps_frame_count / fps_time_diff
                fps_text = f"FPS: {round(fps, 2)}"
                fps_start_time = time.time()
                fps_frame_count = 0
            
            cv2.putText(image, fps_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

def show_fourteenth_window():
    # Close the ninth and tenth windows if they're open
    if 'eleventh_window' in globals():
        eleventh_window.destroy()
    if 'twelveth_window' in globals():
        twelveth_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the fourteenth window variable in the global scope
    global fourteenth_window
    fourteenth_window = tk.Toplevel()
    fourteenth_window.title("fourteenth window")

    # Add UI elements to the eighth_window
    label = tk.Label(fourteenth_window, text="Start Exercise?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the exercise buttons
    button_frame = tk.Frame(fourteenth_window)
    button_frame.pack()

    def start_exercise():
       begin()

    def quit_button():
        # Go back to the second window
        show_second_window()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=start_exercise)
    yes_button.pack(side="left", padx=10)

    # Create the "Quit" button
    quit_button = tk.Button(button_frame, text="Quit", width=10, height=1, command=quit_button)
    quit_button.pack(side="left", padx=10)

def show_thirteenth_window():
    
    # Hide the twelveth window
    twelveth_window.withdraw()

    # Declare the thirteenth window variable in the global scope
    global thirteenth_window
    thirteenth_window = tk.Toplevel()
    thirteenth_window.title("thirteenth window")

    # Add UI elements to the thirteenth window
    label = tk.Label(thirteenth_window, text="Rewatch Demo?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(thirteenth_window)
    button_frame.pack()

    def play_video_again():
        try:
            # Open the video file
            video_file = "core.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        # Hide the twelveth window
                        thirteenth_window.withdraw()
                        show_thirteenth_window()  # Open the thirteenth window again after the video finishes
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def continue_to_therapy():
        # Hide the thirteenth window and show the fourteenth window
        thirteenth_window.withdraw()
        show_fourteenth_window()

    def go_back_to_selection():
        # Hide the thirteenth window and show the main window
        thirteenth_window.withdraw()
        second_window.deiconify()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video_again)
    yes_button.pack(side="left", padx=10)

    # Create the "No, Continue" button
    continue_button = tk.Button(button_frame, text="No, Continue", width=10, height=1, command=continue_to_therapy)
    continue_button.pack(side="left", padx=10)

    # Create the "Back to Therapy Selection" button
    back_button = tk.Button(button_frame, text="Back to Therapy Selection", width=20, height=1, command=go_back_to_selection)
    back_button.pack(side="left", padx=10)

def show_twelveth_window():
    # Close the thirteenth window if it's open
    if 'thirteenth_window' in globals():
        thirteenth_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the sixth_window variable in the global scope
    global twelveth_window
    twelveth_window = tk.Toplevel()
    twelveth_window.title("twelveth window ")

    # Add UI elements to the sixth_window
    label = tk.Label(twelveth_window, text="See Demo for Core Therapy?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(twelveth_window)
    button_frame.pack()

    def play_video():
        try:
            # Open the video file
            video_file = "core.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        show_thirteenth_window()  # Open the thirteenth window after video playback
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video)
    yes_button.pack(side="left", padx=10)

    # Create the "No" button
    no_button = tk.Button(button_frame, text="No", width=10, height=1, command=show_thirteenth_window)
    no_button.pack(side="left", padx=10)

def show_eleventh_window():
    # Close the ninth and tenth windows if they're open
    if 'ninth_window' in globals():
        ninth_window.destroy()
    if 'tenth_window' in globals():
        tenth_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the eighth_window variable in the global scope
    global eleventh_window
    eleventh_window = tk.Toplevel()
    eleventh_window.title("eleventh window")

    # Add UI elements to the eighth_window
    label = tk.Label(eleventh_window, text="Start Exercise?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the exercise buttons
    button_frame = tk.Frame(eleventh_window)
    button_frame.pack()

    def start_exercise():
       begin()

    def quit_button():
        # Go back to the second window
        show_second_window()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=start_exercise)
    yes_button.pack(side="left", padx=10)

    # Create the "Quit" button
    quit_button = tk.Button(button_frame, text="Quit", width=10, height=1, command=quit_button)
    quit_button.pack(side="left", padx=10)
    
def show_tenth_window():
    
    # Hide the ninth window
    ninth_window.withdraw()

    # Declare the tenth_window variable in the global scope
    global tenth_window
    tenth_window = tk.Toplevel()
    tenth_window.title("tenth window")

    # Add UI elements to the seventh window
    label = tk.Label(tenth_window, text="Rewatch Demo?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(tenth_window)
    button_frame.pack()

    def play_video_again():
        try:
            # Open the video file
            video_file = "arm.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        # Hide the third window
                        tenth_window.withdraw()
                        show_tenth_window()  # Open the tenth window again after the video finishes
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def continue_to_therapy():
        # Hide the seventh window and show the eighth window
        tenth_window.withdraw()
        show_eleventh_window()

    def go_back_to_selection():
        # Hide the tenth window and show the main window
        tenth_window.withdraw()
        second_window.deiconify()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video_again)
    yes_button.pack(side="left", padx=10)

    # Create the "No, Continue" button
    continue_button = tk.Button(button_frame, text="No, Continue", width=10, height=1, command=continue_to_therapy)
    continue_button.pack(side="left", padx=10)

    # Create the "Back to Therapy Selection" button
    back_button = tk.Button(button_frame, text="Back to Therapy Selection", width=20, height=1, command=go_back_to_selection)
    back_button.pack(side="left", padx=10)

def show_ninth_window():
    # Close the tenth window if it's open
    if 'tenth_window' in globals():
        tenth_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the sixth_window variable in the global scope
    global ninth_window
    ninth_window = tk.Toplevel()
    ninth_window.title("ninth window ")

    # Add UI elements to the sixth_window
    label = tk.Label(ninth_window, text="See Demo for Arm Therapy?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(ninth_window)
    button_frame.pack()

    def play_video():
        try:
            # Open the video file
            video_file = "arm.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        show_tenth_window()  # Open the tenth window after video playback
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video)
    yes_button.pack(side="left", padx=10)

    # Create the "No" button
    no_button = tk.Button(button_frame, text="No", width=10, height=1, command=show_tenth_window)
    no_button.pack(side="left", padx=10)

def show_eighth_window():
    # Close the sixth and seventh windows if they're open
    if 'sixth_window' in globals():
        sixth_window.destroy()
    if 'seventh_window' in globals():
        seventh_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the eighth_window variable in the global scope
    global eighth_window
    eighth_window = tk.Toplevel()
    eighth_window.title("eighth window")

    # Add UI elements to the eighth_window
    label = tk.Label(eighth_window, text="Start Exercise?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the exercise buttons
    button_frame = tk.Frame(eighth_window)
    button_frame.pack()

    def start_exercise():
       begin()

    def quit_button():
        # Go back to the second window
        show_second_window()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=start_exercise)
    yes_button.pack(side="left", padx=10)

    # Create the "Quit" button
    quit_button = tk.Button(button_frame, text="Quit", width=10, height=1, command=quit_button)
    quit_button.pack(side="left", padx=10)
    
    
def show_seventh_window():
    
    # Hide the sixth window
    sixth_window.withdraw()

    # Declare the seventh_window variable in the global scope
    global seventh_window
    seventh_window = tk.Toplevel()
    seventh_window.title("seventh Window")

    # Add UI elements to the seventh window
    label = tk.Label(seventh_window, text="Rewatch Demo?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(seventh_window)
    button_frame.pack()

    def play_video_again():
        try:
            # Open the video file
            video_file = "leg.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        # Hide the third window
                        seventh_window.withdraw()
                        show_seventh_window()  # Open the seventh window again after the video finishes
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def continue_to_therapy():
        # Hide the seventh window and show the eighth window
        seventh_window.withdraw()
        show_eighth_window()

    def go_back_to_selection():
        # Hide the seventh window and show the main window
        seventh_window.withdraw()
        second_window.deiconify()

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video_again)
    yes_button.pack(side="left", padx=10)

    # Create the "No, Continue" button
    continue_button = tk.Button(button_frame, text="No, Continue", width=10, height=1, command=continue_to_therapy)
    continue_button.pack(side="left", padx=10)

    # Create the "Back to Therapy Selection" button
    back_button = tk.Button(button_frame, text="Back to Therapy Selection", width=20, height=1, command=go_back_to_selection)
    back_button.pack(side="left", padx=10)
    
    
def show_sixth_window():
    # Close the seventh window if it's open
    if 'seventh_window' in globals():
        seventh_window.destroy()

    # Hide the second window
    second_window.withdraw()

    # Declare the sixth_window variable in the global scope
    global sixth_window
    sixth_window = tk.Toplevel()
    sixth_window.title("sixth window ")

    # Add UI elements to the sixth_window
    label = tk.Label(sixth_window, text="See Demo for Leg Therapy?", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the demo buttons
    button_frame = tk.Frame(sixth_window)
    button_frame.pack()

    def play_video():
        try:
            # Open the video file
            video_file = "leg.mp4"  # Replace with the actual video file name
            clip = VideoFileClip(video_file)

            # Create a new window for video playback
            video_window = tk.Toplevel()
            video_window.title("Video Playback")

            # Get the dimensions of the video
            width, height = clip.size

            # Create a canvas to display the video
            canvas = tk.Canvas(video_window, width=width, height=height)
            canvas.pack()

            def update_frame(time):
                try:
                    # Get the current frame of the video
                    frame = clip.get_frame(time)

                    # Convert the frame to an ImageTk.PhotoImage object
                    image = ImageTk.PhotoImage(Image.fromarray(frame))

                    # Update the canvas with the new frame
                    canvas.image = image  # Store the image as an attribute of the canvas
                    canvas.create_image(0, 0, anchor=tk.NW, image=image)

                    # Schedule the next frame update
                    if time + 0.033 < clip.duration:
                        canvas.after(33, update_frame, time + 0.033)
                    else:
                        video_window.destroy()
                        show_seventh_window()  # Open the eighth window after video playback
                except Exception as e:
                    messagebox.showerror("Error", f"An error occurred: {str(e)}")
                    video_window.destroy()

            # Start updating the canvas with video frames
            update_frame(0)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Create the "Yes" button
    yes_button = tk.Button(button_frame, text="Yes", width=10, height=1, command=play_video)
    yes_button.pack(side="left", padx=10)

    # Create the "No" button
    no_button = tk.Button(button_frame, text="No", width=10, height=1, command=show_eighth_window)
    no_button.pack(side="left", padx=10)
    

def show_second_window():
    if 'eighth_window' in globals():
        eighth_window.destroy()
    if 'ninth_window' in globals():
        ninth_window.destroy()
    if 'tenth_window' in globals():
        tenth_window.destroy()
    if 'eleventh_window' in globals():
        eleventh_window.destroy()
    if 'twelveth_window' in globals():
        twelveth_window.destroy()
    if 'thirteenth_window' in globals():
        thirteenth_window.destroy()
    if 'fourteenth_window' in globals():
        fourteenth_window.destroy()

    # Hide the main window
    root.withdraw()

    # Declare the second_window variable in the global scope
    global second_window
    second_window = tk.Toplevel()
    second_window.title("Second Window")

    # Add UI elements to the second window
    label = tk.Label(second_window, text="Select your Therapy", font=("Arial", 16, "bold"))
    label.pack(pady=(50, 20))

    # Create a frame to hold the therapy buttons
    button_frame = tk.Frame(second_window)
    button_frame.pack()

    # Load the images for the therapy buttons
    leg_image = ImageTk.PhotoImage(Image.open("leg_therapy_image.png"))
    back_image = ImageTk.PhotoImage(Image.open("back_therapy_image.png"))
    shoulder_image = ImageTk.PhotoImage(Image.open("shoulder_therapy_image.png"))

    # Create the therapy buttons
    leg_button = tk.Button(button_frame, text="Leg Therapy", compound="top", command=show_sixth_window)
    back_button = tk.Button(button_frame, text="Back Therapy", compound="top", command=show_twelveth_window)
    shoulder_button = tk.Button(button_frame, text="Shoulder Therapy", compound="top", command=show_ninth_window)

    # Set the images for the buttons
    leg_button.config(image=leg_image)
    back_button.config(image=back_image)
    shoulder_button.config(image=shoulder_image)

    # Store the PhotoImage objects as attributes of the buttons
    leg_button.image = leg_image
    back_button.image = back_image
    shoulder_button.image = shoulder_image

    # Configure the buttons
    button_font = font.Font(family="Arial", size=12)
    buttons = [leg_button, back_button, shoulder_button]
    for button in buttons:
        button.configure(font=button_font, bg="black", fg="white", width=150, height=150)
        button.pack(side="left", padx=10, pady=10)

    # Create a button to go back to the main window
    go_back_button = tk.Button(second_window, text="Go Back", command=show_main_window)
    go_back_button.pack(pady=(20, 50))


def show_main_window():
    # Hide the second window
    if 'second_window' in globals():
        second_window.withdraw()

    # Show the main window
    root.deiconify()


def exit_app():
    # Destroy the main window to quit the application
    root.destroy()



# Create the main application window
root = tk.Tk()
root.title("Main Window")

# Load the image file to cover the entire UI page
background_image_path = "background_image.png"  # Replace with the actual path to your image file
background_image = Image.open(background_image_path)
background_photo = ImageTk.PhotoImage(background_image)

# Create a global variable to hold the reference to the PhotoImage object
root.background_photo = background_photo

# Create a label with the background image to cover the entire UI page
background_label = tk.Label(root, image=root.background_photo)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a frame to hold the text and button
frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

# Load the image file to replace the background color of the frame
frame_background_image_path = "frame_background_image.png"  # Replace with the actual path to your image file
frame_background_image = Image.open(frame_background_image_path)
frame_background_photo = ImageTk.PhotoImage(frame_background_image)

# Create a label with the background image for the frame
frame_background_label = tk.Label(frame, image=frame_background_photo)
frame_background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Add UI elements to the frame
welcome_label_font = font.Font(family="Arial", size=16, weight="bold")
welcome_label = tk.Label(frame, text="Welcome to\nGeneral Therapy Assistant App", font=welcome_label_font,
                         bg="black", fg="white")
welcome_label.pack(pady=10)

start_button_font = font.Font(family="Arial", size=16)
start_button = tk.Button(frame, text="Start", font=start_button_font, command=show_second_window, fg="black")
start_button.pack(pady=(0, 10))  # Add some padding at the bottom


# Set the size and position of the main window
window_width = 800
window_height = 600
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_coordinate = int((screen_width / 2) - (window_width / 2))
y_coordinate = int((screen_height / 2) - (window_height / 2))
root.geometry(f"{window_width}x{window_height}+{x_coordinate}+{y_coordinate}")

# Disable window resizing
root.resizable(False, False)

# Bind the exit_app function to the window close button
root.protocol("WM_DELETE_WINDOW", exit_app)

# Start the main event loop
root.mainloop()
